In [2]:
from socket import gethostname
gethostname()

'hpc3-l18-05'

In [3]:
# import os + os.chir is used to get to the llms directory to use the usermodules 
import os
os.chdir('/pub/anvieyra/copilot/llms/user_modules') 

from  table_instances import Summaries, Sentiments, Base
from  db import Session, get_training_data, add_summaries_data
from helper_functions import compute_summary_scores
import pandas as pd
import matplotlib.pyplot as plt

session = Session()

In [4]:
llama3_df = pd.read_csv("../../Datasets/llama3_summary_results.csv")
llama3_df.head()

,dialogue_id,summary,time_taken,memory_usage
0,1,"Mr. Smith visits his doctor, Doctor Hawkins, f...",0.830272,277.273711
1,2,Dr. Peters and Mrs. Parker are at the doctor's...,0.830643,261.101680
2,3,Person1 asks Person2 if they saw a set of keys...,0.830643,261.101680
3,4,Person1 is upset because Person2 didn't tell t...,0.830643,261.101680
4,5,Malik approaches Wen and Nikki at a party and ...,0.830643,261.101680


In [5]:
training_data = get_training_data()
training_data.head()

,dialogue_id,dataset,dialogue_text,actual_summary,actual_sentiment
0,3816,training,"#Person1#: Good morning, sir. Is there anythin...",#Person2# wants a suit made in 100 % black Bar...,neutral
1,3,training,"#Person1#: Excuse me, did you see a set of key...",#Person1#'s looking for a set of keys and asks...,positive
2,4,training,#Person1#: Why didn't you tell me you had a gi...,#Person1#'s angry because #Person2# didn't tel...,negative
3,5,training,"#Person1#: Watsup, ladies! Y'll looking'fine t...",Malik invites Nikki to dance. Nikki agrees if ...,positive
4,6,training,"#Person1#: Happy birthday, Aims!\n#Person2#: T...",Lisa gives Aims a birthday present and Aims lo...,positive


In [6]:
merged_df = pd.merge(training_data[["dialogue_id","actual_summary"]], llama3_df, on="dialogue_id", how='inner')
merged_df.head()

,dialogue_id,actual_summary,summary,time_taken,memory_usage
0,3816,#Person2# wants a suit made in 100 % black Bar...,Person 1 is a tailor who is asked by Person 2 ...,0.830643,261.10168
1,3,#Person1#'s looking for a set of keys and asks...,Person1 asks Person2 if they saw a set of keys...,0.830643,261.10168
2,4,#Person1#'s angry because #Person2# didn't tel...,Person1 is upset because Person2 didn't tell t...,0.830643,261.10168
3,5,Malik invites Nikki to dance. Nikki agrees if ...,Malik approaches Wen and Nikki at a party and ...,0.830643,261.10168
4,6,Lisa gives Aims a birthday present and Aims lo...,"Person1 gives Person2 a birthday present, whic...",0.830643,261.10168


In [7]:
merged_df = merged_df.rename(columns={'summary': 'generated_summary', 'time_taken':'time_summary_taken', 'memory_usage':'memory_summary_usage'})
merged_df.head()

,dialogue_id,actual_summary,generated_summary,time_summary_taken,memory_summary_usage
0,3816,#Person2# wants a suit made in 100 % black Bar...,Person 1 is a tailor who is asked by Person 2 ...,0.830643,261.10168
1,3,#Person1#'s looking for a set of keys and asks...,Person1 asks Person2 if they saw a set of keys...,0.830643,261.10168
2,4,#Person1#'s angry because #Person2# didn't tel...,Person1 is upset because Person2 didn't tell t...,0.830643,261.10168
3,5,Malik invites Nikki to dance. Nikki agrees if ...,Malik approaches Wen and Nikki at a party and ...,0.830643,261.10168
4,6,Lisa gives Aims a birthday present and Aims lo...,"Person1 gives Person2 a birthday present, whic...",0.830643,261.10168


In [8]:
merged_df["model_id"] = "llama 3 8b"
merged_df.head()

,dialogue_id,actual_summary,generated_summary,time_summary_taken,memory_summary_usage,model_id
0,3816,#Person2# wants a suit made in 100 % black Bar...,Person 1 is a tailor who is asked by Person 2 ...,0.830643,261.10168,llama 3 8b
1,3,#Person1#'s looking for a set of keys and asks...,Person1 asks Person2 if they saw a set of keys...,0.830643,261.10168,llama 3 8b
2,4,#Person1#'s angry because #Person2# didn't tel...,Person1 is upset because Person2 didn't tell t...,0.830643,261.10168,llama 3 8b
3,5,Malik invites Nikki to dance. Nikki agrees if ...,Malik approaches Wen and Nikki at a party and ...,0.830643,261.10168,llama 3 8b
4,6,Lisa gives Aims a birthday present and Aims lo...,"Person1 gives Person2 a birthday present, whic...",0.830643,261.10168,llama 3 8b


In [9]:
import logging
logging.getLogger("transformers").setLevel(logging.ERROR)

merged_df = merged_df.apply(compute_summary_scores, axis=1)
merged_df.head()

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

,dialogue_id,actual_summary,generated_summary,time_summary_taken,memory_summary_usage,model_id,rouge_1,rouge_2,rouge_l,bert_precision,bert_recall,bert_f1,meteor
0,3816,#Person2# wants a suit made in 100 % black Bar...,Person 1 is a tailor who is asked by Person 2 ...,0.830643,261.10168,llama 3 8b,0.370370,0.126984,0.370370,0.873162,0.919966,0.895953,0.554318
1,3,#Person1#'s looking for a set of keys and asks...,Person1 asks Person2 if they saw a set of keys...,0.830643,261.10168,llama 3 8b,0.408163,0.111111,0.285714,0.880703,0.876892,0.878794,0.409627
2,4,#Person1#'s angry because #Person2# didn't tel...,Person1 is upset because Person2 didn't tell t...,0.830643,261.10168,llama 3 8b,0.264151,0.030303,0.226415,0.877741,0.872372,0.875048,0.257764
3,5,Malik invites Nikki to dance. Nikki agrees if ...,Malik approaches Wen and Nikki at a party and ...,0.830643,261.10168,llama 3 8b,0.196078,0.057143,0.196078,0.870300,0.885064,0.877620,0.213675
4,6,Lisa gives Aims a birthday present and Aims lo...,"Person1 gives Person2 a birthday present, whic...",0.830643,261.10168,llama 3 8b,0.285714,0.086957,0.285714,0.885728,0.906695,0.896089,0.381197


In [10]:
merged_df.to_csv("../../Datasets/llama3_scores.csv", index = False)


In [11]:
llama3_scores = pd.read_csv("../../Datasets/llama3_scores.csv")
llama3_scores.head()

,dialogue_id,actual_summary,generated_summary,time_summary_taken,memory_summary_usage,model_id,rouge_1,rouge_2,rouge_l,bert_precision,bert_recall,bert_f1,meteor
0,3816,#Person2# wants a suit made in 100 % black Bar...,Person 1 is a tailor who is asked by Person 2 ...,0.830643,261.10168,llama 3 8b,0.370370,0.126984,0.370370,0.873162,0.919966,0.895953,0.554318
1,3,#Person1#'s looking for a set of keys and asks...,Person1 asks Person2 if they saw a set of keys...,0.830643,261.10168,llama 3 8b,0.408163,0.111111,0.285714,0.880703,0.876892,0.878794,0.409627
2,4,#Person1#'s angry because #Person2# didn't tel...,Person1 is upset because Person2 didn't tell t...,0.830643,261.10168,llama 3 8b,0.264151,0.030303,0.226415,0.877741,0.872372,0.875048,0.257764
3,5,Malik invites Nikki to dance. Nikki agrees if ...,Malik approaches Wen and Nikki at a party and ...,0.830643,261.10168,llama 3 8b,0.196078,0.057143,0.196078,0.870300,0.885064,0.877620,0.213675
4,6,Lisa gives Aims a birthday present and Aims lo...,"Person1 gives Person2 a birthday present, whic...",0.830643,261.10168,llama 3 8b,0.285714,0.086957,0.285714,0.885728,0.906695,0.896089,0.381197


In [12]:
merged_df = merged_df[['dialogue_id', 'model_id', 'generated_summary', 'rouge_1', 'rouge_2', 'rouge_l', 'bert_precision', 'bert_recall', 'bert_f1', 'meteor', 'memory_summary_usage', 'time_summary_taken']]
merged_df.head()

,dialogue_id,model_id,generated_summary,rouge_1,rouge_2,rouge_l,bert_precision,bert_recall,bert_f1,meteor,memory_summary_usage,time_summary_taken
0,3816,llama 3 8b,Person 1 is a tailor who is asked by Person 2 ...,0.370370,0.126984,0.370370,0.873162,0.919966,0.895953,0.554318,261.10168,0.830643
1,3,llama 3 8b,Person1 asks Person2 if they saw a set of keys...,0.408163,0.111111,0.285714,0.880703,0.876892,0.878794,0.409627,261.10168,0.830643
2,4,llama 3 8b,Person1 is upset because Person2 didn't tell t...,0.264151,0.030303,0.226415,0.877741,0.872372,0.875048,0.257764,261.10168,0.830643
3,5,llama 3 8b,Malik approaches Wen and Nikki at a party and ...,0.196078,0.057143,0.196078,0.870300,0.885064,0.877620,0.213675,261.10168,0.830643
4,6,llama 3 8b,"Person1 gives Person2 a birthday present, whic...",0.285714,0.086957,0.285714,0.885728,0.906695,0.896089,0.381197,261.10168,0.830643


In [13]:
add_summaries_data(merged_df)